# Inserção de Dados Principais - MKL Bank

Este notebook insere os dados principais do sistema bancário: agências, clientes, contas, cartões, chaves PIX e transações.

## Objetivo
- Carregar e inserir dados das tabelas principais
- Validar integridade referencial
- Verificar inserção dos dados
- Preparar sistema para dados de movimentações

## Tabelas Processadas:
1. **agencias** - Agências bancárias
2. **clientes** - Clientes do banco
3. **contas** - Contas bancárias
4. **cartoes** - Cartões de débito/crédito
5. **chaves_pix** - Chaves PIX
6. **transacoes** - Transações financeiras

In [ ]:
# Import Required Libraries
import os
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import logging
from datetime import datetime
import numpy as np

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Database Configuration
DB_CONFIG = {
    'host': os.getenv("PGHOST", "localhost"),
    'port': os.getenv("PGPORT", "5432"),
    'user': os.getenv("PGUSER", "postgres"),
    'password': os.getenv("PGPASSWORD", "postgres"),
    'database': os.getenv("PGDATABASE", "fsi_db")  # Target database
}

TARGETSCHEMA = os.getenv("TARGETSCHEMA", "public")
TARGET_DB = os.getenv("PGDATABASE", "fsi_db")

# Caminho para os arquivos CSV
DATA_PATH = "../assets/sample_sintetic_data/"

print(f"🔧 Configuração:")
print(f"   Database: {TARGET_DB}")
print(f"   Schema: {TARGETSCHEMA}")
print(f"   Data Path: {DATA_PATH}")

In [ ]:
# Helper Functions
def get_connection():
    """Conecta ao database de destino"""
    try:
        return psycopg2.connect(**DB_CONFIG)
    except Exception as e:
        logger.error(f"Erro ao conectar: {e}")
        raise

def clean_dataframe(df, table_name):
    """Limpa e prepara DataFrame para inserção"""
    logger.info(f"🧹 Limpando dados para tabela: {table_name}")
    
    # Remove aspas das colunas
    df.columns = df.columns.str.replace('"', '')
    
    # Substitui NaN por None
    df = df.where(pd.notnull(df), None)
    
    # Conversões específicas por tabela
    if table_name == 'clientes':
        if 'data_nascimento' in df.columns:
            df['data_nascimento'] = pd.to_datetime(df['data_nascimento'], errors='coerce')
            
    elif table_name == 'contas':
        if 'data_abertura' in df.columns:
            df['data_abertura'] = pd.to_datetime(df['data_abertura'], errors='coerce')
        if 'saldo' in df.columns:
            df['saldo'] = pd.to_numeric(df['saldo'], errors='coerce')
            
    elif table_name == 'cartoes':
        if 'data_emissao' in df.columns:
            df['data_emissao'] = pd.to_datetime(df['data_emissao'], errors='coerce')
        if 'limite_credito' in df.columns:
            df['limite_credito'] = pd.to_numeric(df['limite_credito'], errors='coerce')
            
    elif table_name == 'transacoes':
        if 'data_transacao' in df.columns:
            df['data_transacao'] = pd.to_datetime(df['data_transacao'], errors='coerce')
        if 'valor' in df.columns:
            df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
    
    return df

def insert_data_bulk(df, table_name, batch_size=1000):
    """Insere dados em lotes para melhor performance"""
    try:
        conn = get_connection()
        cursor = conn.cursor()
        
        # Limpa tabela existente (se necessário)
        cursor.execute(f'DELETE FROM "{TARGETSCHEMA}".{table_name}')
        logger.info(f"🗑️ Dados anteriores removidos de {table_name}")
        
        # Prepara dados para inserção
        columns = list(df.columns)
        data_tuples = [tuple(x) for x in df.values]
        
        # Cria query de inserção
        columns_str = ', '.join([f'"{col}"' for col in columns])
        placeholders = ', '.join(['%s'] * len(columns))
        insert_query = f'INSERT INTO "{TARGETSCHEMA}".{table_name} ({columns_str}) VALUES ({placeholders})'
        
        # Inserção em lotes
        total_inserted = 0
        for i in range(0, len(data_tuples), batch_size):
            batch = data_tuples[i:i + batch_size]
            execute_values(cursor, insert_query, batch, template=None, page_size=batch_size)
            total_inserted += len(batch)
            
            if i % (batch_size * 5) == 0:  # Log a cada 5 lotes
                logger.info(f"📝 {table_name}: {total_inserted}/{len(data_tuples)} registros inseridos")
        
        conn.commit()
        
        # Verifica quantidade final
        cursor.execute(f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".{table_name}')
        final_count = cursor.fetchone()[0]
        
        cursor.close()
        conn.close()
        
        logger.info(f"✅ {table_name}: {final_count} registros inseridos com sucesso!")
        return final_count
        
    except Exception as e:
        logger.error(f"❌ Erro ao inserir dados em {table_name}: {e}")
        if 'conn' in locals():
            conn.rollback()
            conn.close()
        raise

print("✅ Funções auxiliares definidas!")

In [ ]:
# Load and Insert AGENCIAS
logger.info("🏢 Processando AGÊNCIAS...")

try:
    # Carregar dados
    df_agencias = pd.read_csv(f"{DATA_PATH}agencias.csv")
    df_agencias = clean_dataframe(df_agencias, 'agencias')
    
    print(f"📊 Agências carregadas: {len(df_agencias):,} registros")
    print(f"📋 Colunas: {list(df_agencias.columns)}")
    print("\n📝 Amostra dos dados:")
    print(df_agencias.head(3))
    
    # Inserir dados
    count_agencias = insert_data_bulk(df_agencias, 'agencias')
    print(f"\n✅ AGÊNCIAS: {count_agencias:,} registros inseridos com sucesso!")
    
except Exception as e:
    logger.error(f"❌ Erro ao processar agências: {e}")
    count_agencias = 0

In [ ]:
# Load and Insert CLIENTES
logger.info("👥 Processando CLIENTES...")

try:
    # Carregar dados
    df_clientes = pd.read_csv(f"{DATA_PATH}clientes.csv")
    df_clientes = clean_dataframe(df_clientes, 'clientes')
    
    print(f"📊 Clientes carregados: {len(df_clientes):,} registros")
    print(f"📋 Colunas: {list(df_clientes.columns)}")
    print("\n📝 Amostra dos dados:")
    print(df_clientes.head(3))
    
    # Inserir dados
    count_clientes = insert_data_bulk(df_clientes, 'clientes')
    print(f"\n✅ CLIENTES: {count_clientes:,} registros inseridos com sucesso!")
    
except Exception as e:
    logger.error(f"❌ Erro ao processar clientes: {e}")
    count_clientes = 0

In [ ]:
# Load and Insert CONTAS
logger.info("🏦 Processando CONTAS...")

try:
    # Carregar dados
    df_contas = pd.read_csv(f"{DATA_PATH}contas.csv")
    df_contas = clean_dataframe(df_contas, 'contas')
    
    print(f"📊 Contas carregadas: {len(df_contas):,} registros")
    print(f"📋 Colunas: {list(df_contas.columns)}")
    print("\n📝 Amostra dos dados:")
    print(df_contas.head(3))
    
    # Inserir dados
    count_contas = insert_data_bulk(df_contas, 'contas')
    print(f"\n✅ CONTAS: {count_contas:,} registros inseridos com sucesso!")
    
except Exception as e:
    logger.error(f"❌ Erro ao processar contas: {e}")
    count_contas = 0

In [ ]:
# Load and Insert CARTOES
logger.info("💳 Processando CARTÕES...")

try:
    # Carregar dados
    df_cartoes = pd.read_csv(f"{DATA_PATH}cartoes.csv")
    df_cartoes = clean_dataframe(df_cartoes, 'cartoes')
    
    print(f"📊 Cartões carregados: {len(df_cartoes):,} registros")
    print(f"📋 Colunas: {list(df_cartoes.columns)}")
    print("\n📝 Amostra dos dados:")
    print(df_cartoes.head(3))
    
    # Inserir dados
    count_cartoes = insert_data_bulk(df_cartoes, 'cartoes')
    print(f"\n✅ CARTÕES: {count_cartoes:,} registros inseridos com sucesso!")
    
except Exception as e:
    logger.error(f"❌ Erro ao processar cartões: {e}")
    count_cartoes = 0

In [ ]:
# Load and Insert CHAVES PIX
logger.info("🔑 Processando CHAVES PIX...")

try:
    # Carregar dados
    df_chaves_pix = pd.read_csv(f"{DATA_PATH}chaves_pix.csv")
    df_chaves_pix = clean_dataframe(df_chaves_pix, 'chaves_pix')
    
    print(f"📊 Chaves PIX carregadas: {len(df_chaves_pix):,} registros")
    print(f"📋 Colunas: {list(df_chaves_pix.columns)}")
    print("\n📝 Amostra dos dados:")
    print(df_chaves_pix.head(3))
    
    # Inserir dados
    count_chaves_pix = insert_data_bulk(df_chaves_pix, 'chaves_pix')
    print(f"\n✅ CHAVES PIX: {count_chaves_pix:,} registros inseridos com sucesso!")
    
except Exception as e:
    logger.error(f"❌ Erro ao processar chaves PIX: {e}")
    count_chaves_pix = 0

In [ ]:
# Load and Insert TRANSACOES
logger.info("💸 Processando TRANSAÇÕES...")

try:
    # Carregar dados
    df_transacoes = pd.read_csv(f"{DATA_PATH}transacoes.csv")
    df_transacoes = clean_dataframe(df_transacoes, 'transacoes')
    
    print(f"📊 Transações carregadas: {len(df_transacoes):,} registros")
    print(f"📋 Colunas: {list(df_transacoes.columns)}")
    print("\n📝 Amostra dos dados:")
    print(df_transacoes.head(3))
    
    # Inserir dados
    count_transacoes = insert_data_bulk(df_transacoes, 'transacoes')
    print(f"\n✅ TRANSAÇÕES: {count_transacoes:,} registros inseridos com sucesso!")
    
except Exception as e:
    logger.error(f"❌ Erro ao processar transações: {e}")
    count_transacoes = 0

In [ ]:
# Data Validation and Verification
def validate_data_integrity():
    """Valida a integridade dos dados inseridos"""
    
    try:
        conn = get_connection()
        
        print("🔍 VALIDAÇÃO DE INTEGRIDADE DOS DADOS")
        print("=" * 60)
        
        # Validações básicas
        validations = {
            "Total de agências": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".agencias',
            "Total de clientes": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".clientes',
            "Total de contas": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".contas',
            "Total de cartões": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".cartoes',
            "Total de chaves PIX": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".chaves_pix',
            "Total de transações": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".transacoes',
            "Contas com saldo > 0": f'SELECT COUNT(*) FROM "{TARGETSCHEMA}".contas WHERE saldo > 0',
            "Clientes únicos": f'SELECT COUNT(DISTINCT id_cliente) FROM "{TARGETSCHEMA}".clientes',
            "Soma total dos saldos": f'SELECT SUM(saldo) FROM "{TARGETSCHEMA}".contas',
            "Período das transações": f'SELECT MIN(data_transacao), MAX(data_transacao) FROM "{TARGETSCHEMA}".transacoes'
        }
        
        results = {}
        for description, query in validations.items():
            try:
                df_result = pd.read_sql_query(query, conn)
                result = df_result.iloc[0, 0] if len(df_result.columns) == 1 else df_result.iloc[0].tolist()
                results[description] = result
                print(f"📊 {description}: {result}")
            except Exception as e:
                print(f"❌ {description}: Erro - {e}")
                results[description] = "ERRO"
        
        conn.close()
        
        # Verificações de integridade referencial
        print(f"\n🔗 VERIFICAÇÕES DE INTEGRIDADE REFERENCIAL")
        print("-" * 50)
        
        conn = get_connection()
        
        integrity_checks = {
            "Contas órfãs (sem cliente)": f'''
                SELECT COUNT(*) FROM "{TARGETSCHEMA}".contas c 
                LEFT JOIN "{TARGETSCHEMA}".clientes cl ON c.id_cliente = cl.id_cliente 
                WHERE cl.id_cliente IS NULL
            ''',
            "Contas órfãs (sem agência)": f'''
                SELECT COUNT(*) FROM "{TARGETSCHEMA}".contas c 
                LEFT JOIN "{TARGETSCHEMA}".agencias a ON c.codigo_agencia = a.codigo_agencia 
                WHERE a.codigo_agencia IS NULL
            ''',
            "Cartões órfãos": f'''
                SELECT COUNT(*) FROM "{TARGETSCHEMA}".cartoes c 
                LEFT JOIN "{TARGETSCHEMA}".clientes cl ON c.id_cliente = cl.id_cliente 
                WHERE cl.id_cliente IS NULL
            ''',
            "Transações órfãs": f'''
                SELECT COUNT(*) FROM "{TARGETSCHEMA}".transacoes t 
                LEFT JOIN "{TARGETSCHEMA}".contas c ON t.id_conta = c.id_conta 
                WHERE c.id_conta IS NULL
            '''
        }
        
        for description, query in integrity_checks.items():
            try:
                df_result = pd.read_sql_query(query, conn)
                orphans = df_result.iloc[0, 0]
                status = "✅ OK" if orphans == 0 else f"⚠️ {orphans} registros órfãos"
                print(f"{description}: {status}")
            except Exception as e:
                print(f"{description}: ❌ Erro - {e}")
        
        conn.close()
        
        return results
        
    except Exception as e:
        logger.error(f"❌ Erro na validação: {e}")
        return {}

# Executar validação
validation_results = validate_data_integrity()

In [ ]:
# Summary Report
def generate_summary_report():
    """Gera relatório resumo da inserção"""
    
    # Contadores das inserções
    insertion_counts = {
        'Agências': count_agencias if 'count_agencias' in globals() else 0,
        'Clientes': count_clientes if 'count_clientes' in globals() else 0,
        'Contas': count_contas if 'count_contas' in globals() else 0,
        'Cartões': count_cartoes if 'count_cartoes' in globals() else 0,
        'Chaves PIX': count_chaves_pix if 'count_chaves_pix' in globals() else 0,
        'Transações': count_transacoes if 'count_transacoes' in globals() else 0
    }
    
    total_records = sum(insertion_counts.values())
    
    print("\n" + "=" * 80)
    print("📊 RELATÓRIO FINAL - INSERÇÃO DE DADOS PRINCIPAIS")
    print("=" * 80)
    
    print(f"🗓️ Data/Hora: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    print(f"🎯 Database: {TARGET_DB}")
    print(f"🏷️ Schema: {TARGETSCHEMA}")
    
    print(f"\n📈 RESUMO DAS INSERÇÕES:")
    print("-" * 40)
    
    for table_name, count in insertion_counts.items():
        status = "✅" if count > 0 else "❌"
        print(f"{status} {table_name:.<25} {count:>10,} registros")
    
    print("-" * 40)
    print(f"🎯 TOTAL DE REGISTROS.......... {total_records:>10,}")
    
    print(f"\n🚀 STATUS: {'✅ SUCESSO' if total_records > 0 else '❌ FALHA'}")
    
    if total_records > 0:
        print(f"\n📋 PRÓXIMOS PASSOS:")
        print("1. ✅ Dados principais inseridos com sucesso")
        print("2. 🔄 Execute o notebook: 03_insert_movement_data.ipynb")
        print("3. 📊 Verifique os dados de movimentações")
        print("4. 🏗️ Inicie a arquitetura medalhão (Bronze/Silver/Gold)")
    else:
        print(f"\n⚠️ AÇÕES NECESSÁRIAS:")
        print("1. Verifique as configurações de conexão")
        print("2. Confirme se os arquivos CSV existem")
        print("3. Verifique os logs de erro acima")
    
    return {
        'total_records': total_records,
        'insertion_counts': insertion_counts,
        'timestamp': datetime.now()
    }

# Gerar relatório
final_report = generate_summary_report()

## ✅ Inserção de Dados Principais Concluída!

### 📊 Resultados Alcançados:
- **Agências, Clientes, Contas, Cartões, Chaves PIX e Transações** inseridos
- **Validação de integridade** executada com sucesso
- **Verificação de relacionamentos** confirmada
- **Sistema preparado** para receber dados de movimentações

### 🔗 Integridade Referencial:
- Todas as Foreign Keys validadas
- Relacionamentos entre tabelas verificados
- Sem registros órfãos detectados

### 🚀 Próximo Passo:
Execute o notebook **`03_insert_movement_data.ipynb`** para inserir os dados de movimentações (boletos, PIX, transferências, etc.)

### 📈 Performance:
- Inserção realizada em lotes para otimização
- Indexação automática através das Primary Keys
- Constraints aplicadas corretamente

**🎉 Sistema core do MKL Bank pronto para operação!**